## Data flow to mysql

In [1]:
import mysql.connector as mdb
from mysql.connector import Error
import sys, traceback

In [2]:
def open_database(host, db_name, username, password):
    try:
        connection = mdb.connect(host=host,
                                 database=db_name,
                                 user=username,
                                 password=password)
        if connection.is_connected():
           db_Info = connection.get_server_info()
           print("Connected to MySQL database... MySQL Server version on ",db_Info)
           cursor = connection.cursor()
           cursor.execute("select database();")
           record = cursor.fetchone()
           print ("Your connected to - ", record)
           cursor.close()
    except Error as e :
        print ("Error while connecting to MySQL", e)
    return connection

In [3]:
def close_database(connection):
    #closing database connection.
    if(connection.is_connected()):
        connection.close()
        print("MySQL connection is closed")

In [4]:
conn = open_database('localhost', 'caselaw', 'root', PASSWORD)

Connected to MySQL database... MySQL Server version on  8.0.20
Your connected to -  ('caselaw',)


In [5]:
query = """SELECT * FROM caselaw.case LIMIT 1;"""
cursor = conn.cursor(buffered=True)
cursor.execute(query)
records = cursor.fetchall()
for row in records:
    print(row)

(1, datetime.date(2000, 1, 11), 'NULL', 'NULL', 'NL', 'NULL', 'NULL', 'NULL', datetime.date(1900, 1, 1), 'NULL', 'ECLI:NL:CBB:2000:ZG2046', 221)


---
### Datasets

In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [7]:
!ls ../data_dump

case.csv                               cases_metadata.csv
case_opinion_from_advocate_general.csv countries.csv
caselaw.zip                            legal_intelligence_cases.csv
caselawDB.sql                          legislation_citations.csv
caselaw_citations.csv                  sample
caselaw_db.sql                         test.csv


In [8]:
df_case_raw = pd.read_csv("../data_dump/case.csv").replace(np.nan, 'NULL', regex=True)
df_country = pd.read_csv("../data_dump/countries.csv").replace(np.nan, 'NULL', regex=True)
df_li_cases = pd.read_csv("../data_dump/legal_intelligence_cases.csv").replace(np.nan, 'NULL', regex=True).drop_duplicates(subset ="ecli")
df_case_opinion = pd.read_csv("../data_dump/case_opinion_from_advocate_general.csv").replace(np.nan, 'NULL', regex=True)
df_case_citation = pd.read_csv("../data_dump/caselaw_citations.csv").replace(np.nan, 'NULL', regex=True)
df_legislation_citation = pd.read_csv("../data_dump/legislation_citations.csv").replace(np.nan, 'NULL', regex=True)

In [9]:
#change LI Ecli to not underscores
for i, li_case in df_li_cases.iterrows():
    #get the ecli number
    ecli = li_case.ecli
    #replace the underscores
    new_ecli = ecli.replace('_', ':')
    #save the new ecli in the dataframe
    df_li_cases.at[i, 'ecli'] = new_ecli

---
### Sample to run the procedure

In [12]:
#taking recent cases to get important links
df_case = df_case_raw[df_case_raw['date'] > '2000-01-01 00:00:00']

sample_size = 400000
#list of ecli numbers we choose for our sample
sample = list(df_case['case_id'].sample(n=sample_size, random_state=18))

#compare case opinion and cases on ecli cause it seems like there is no overlap 
#case_eclis = df_case['case_id'].values
#opinion_eclis = df_case_opinion['case_id'].values

##0: case sample
df_case = df_case[df_case['case_id'].isin(sample)].reset_index(drop=True)

##1: legislation citations sample
df_legislation_citation = df_legislation_citation[df_legislation_citation['source_ecli'].isin(sample)].reset_index(drop=True)

##2: case citations sample
df_case_citation = df_case_citation[df_case_citation['source_ecli'].isin(sample)].reset_index(drop=True)

##3: case opinions sample
df_case_opinion = df_case_opinion[df_case_opinion['case_id'].isin(sample)].reset_index(drop=True)

##4: li cases sample
df_li_cases = df_li_cases[df_li_cases['ecli'].isin(sample)].reset_index(drop=True)

print('Number of Cases in\nCases: {}\nLegislation citation: {}\nCase citation: {}\nCase opinion: {}\nLI cases: {}'\
      .format(len(df_case), len(df_legislation_citation), len(df_case_citation), len(df_case_opinion), len(df_li_cases)))

Number of Cases in
Cases: 400000
Legislation citation: 0
Case citation: 0
Case opinion: 0
LI cases: 13


In [88]:
df_case_opinion[df_case_opinion['case_id'].isin(sample)].reset_index(drop=True)

,case_id,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,...,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text


In [12]:
df_case.head()

,case_id,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,...,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text
0,ECLI:NL:CBB:2000:ZG2046,2000-01-11,95/0775/062/229,NULL,NL,NULL,NULL,NULL,NULL,NL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,ECLI:NL:CBB:2000:ZG1822,2000-02-15,98/280;20100,NULL,NL,NULL,NULL,NULL,NULL,NL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,ECLI:NL:CRVB:2000:AA5272,2000-02-04,97/8189 AAW/WAO,"\n -<para xmlns=""http://www.rechtspraak.n...",NL,NULL,NULL,Hoger beroep,NULL,NL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,ECLI:NL:CRVB:2000:AI5503,2000-01-20,97/11168 WUBO,NULL,NL,NULL,NULL,NULL,NULL,NL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,ECLI:NL:CRVB:2000:AA5886,2000-02-04,97/173 AAW/WAO,"\n -<para xmlns=""http://www.rechtspraak.n...",NL,NULL,NULL,Hoger beroep,NULL,NL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [13]:
#pd.Series(df_case.case_id).to_csv('eclis_sample5k.csv', index=False)

---
### Utils

In [14]:
def to_tuples(df):
    return [tuple(x) for x in df.values]

In [15]:
def clean_table_sql(table_name):
    try:
        cursor1 = conn.cursor(buffered=True)
        query1 = """DELETE FROM `{}`;""".format(table_name)
        cursor1.execute(query1)
        cursor2 = conn.cursor(buffered=True)
        query2 = """ALTER TABLE `{}` AUTO_INCREMENT = 1;""".format(table_name)
        cursor2.execute(query2)
        conn.commit()
    except Error as error :
        conn.rollback()
        print("Failed to delete MySQL table {}".format(error))
        traceback.print_exc(file=sys.stdout)

In [16]:
def get_parent_ids(table, column_table, df, column_df):    
    """DB table, DB column_table, df: pandas df to look at, column_df"""
    pid = []
    cursor = conn.cursor()
    for idx, data in enumerate(df[column_df]):
        if "'" in data:
            query = """SELECT id FROM `{}` WHERE {} = \"{}\" """.format(table, column_table, data)
        else:
            query = """SELECT id FROM `{}` WHERE {} = '{}' """.format(table, column_table, data)
        #print(idx, data)
        cursor.execute(query)
        records = cursor.fetchall()
        if(len(records) == 0):
            print('select did not find match')
        pid.append(records[0][0])
    cursor.close()
    return pid

---
### Tables Hierarchies

1. Court
2. Case
3. Case Opinion
4. LI Case
5. Subject
6. Case subject
7. Country
8. Case country
9. Case citation
10. Legislation citation
11. Case related decision


In [17]:
clean_table_sql('case_related_decision')
clean_table_sql('legislation_citation')
clean_table_sql('case_citation')
clean_table_sql('case_country')
clean_table_sql('country')
clean_table_sql('case_subject')
clean_table_sql('subject')
clean_table_sql('legal_intelligence_case')
clean_table_sql('case_opinion')
clean_table_sql('case')
clean_table_sql('court')

---
### Courts

In [18]:
court = pd.DataFrame()

In [19]:
courts_list = df_case_raw.authority.unique()
courts_list = [i.replace('"','-') for i in courts_list]
court['name'] = courts_list

In [20]:
#dummies
court.loc[:,'type'] = 'NULL'
court.loc[:,'level'] = 'NULL'
court.loc[:,'country'] = 'NULL'
court.loc[:,'language'] = 'NULL' 
court.loc[:,'jurisdiction'] = 'NULL' 
court.loc[:,'law_area'] = 'NULL'
court.loc[:,'authority_level'] = 'NULL'

In [21]:
#court.loc[810,'name']#.head(2)

In [22]:
tuples = to_tuples(court)

In [23]:
#(\"%s\", %s, %s, %s, %s, %s, %s, %s)")
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `court`"
                "(name, type, level, country, language, jurisdiction, law_area, authority_level)"
                "VALUES (\"%s\", %s, %s, %s, %s, %s, %s, %s)")%data
        #print(idx, query)
        cursor.execute(query)
    conn.commit()
    cursor.close()
    print('court added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

court added


In [24]:
len(court)

1223

---
### Case

In [25]:
case = pd.DataFrame()
case_columns = ['date',
        'description',
        'language',
        'venue',
        'abstract',
        'procedure_type',
        'lodge_date',
        'link',
        'ecli']

In [26]:
temp = df_case[['date',
                'description',
                'language',
                'venue',
                'abstract',
                'procedure_type',
                'lodge_date',
                'alternative_sources',
                'case_id']]
temp.columns = case_columns
case = case.append(temp, ignore_index=True)

In [27]:
parents_ids = get_parent_ids('court', 'name', df_case, 'authority')

In [28]:
def get_parent_ids(table, column_table, df, column_df):    
    """DB table, DB column_table, df: pandas df to look at, column_df"""
    pid = []
    cursor = conn.cursor()
    for idx, data in enumerate(df[column_df]):
        if "'" in data:
            query = """SELECT id FROM `{}` WHERE {} = \"{}\" """.format(table, column_table, data)
        else:
            query = """SELECT id FROM `{}` WHERE {} = '{}' """.format(table, column_table, data)
        #print(idx, data)
        cursor.execute(query)
        records = cursor.fetchall()
        if(len(records) == 0):
            print('select did not find match')
        pid.append(records[0][0])
    cursor.close()
    return pid

In [29]:
case['name'] = 'NULL'
case['court_id'] = parents_ids
case['date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case['date']]
case['lodge_date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case['lodge_date']]

In [30]:
case.head()

,date,description,language,venue,abstract,procedure_type,lodge_date,link,ecli,name,court_id
0,2000-01-11,NULL,NL,NULL,NULL,NULL,1900-01-01,NULL,ECLI:NL:CBB:2000:ZG2046,NULL,221
1,2000-02-15,NULL,NL,NULL,NULL,NULL,1900-01-01,NULL,ECLI:NL:CBB:2000:ZG1822,NULL,221
2,2000-02-04,"\n -<para xmlns=""http://www.rechtspraak.n...",NL,NULL,NULL,Hoger beroep,1900-01-01,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:CRVB:2000:AA5272,NULL,129
3,2000-01-20,NULL,NL,NULL,NULL,NULL,1900-01-01,NULL,ECLI:NL:CRVB:2000:AI5503,NULL,129
4,2000-02-04,"\n -<para xmlns=""http://www.rechtspraak.n...",NL,NULL,NULL,Hoger beroep,1900-01-01,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:CRVB:2000:AA5886,NULL,129


In [31]:
tuples = to_tuples(case)

In [32]:
try:  
    cursor = conn.cursor(buffered=True)
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `case`"
                "(date, description, language, venue, abstract, procedure_type, lodge_date, link, ecli, name, court_id)"
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
        cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('case added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

case added


---
### Case opinion advocate general

In [33]:
case_opinion = pd.DataFrame()
case_opinion_columns = ['date',
                        'case_number',
                        'description',
                        'language',
                        'country',
                        'venue',
                        'abstract',
                        'procedure_type',
                        'authority',
                        'ecli']

In [34]:
temp = df_case_opinion[['date',
                        'case_number',
                        'description',
                        'language',
                        'country',
                        'venue',
                        'abstract',
                        'procedure_type',
                        'authority',
                        'case_id']]
temp.columns = case_opinion_columns
case_opinion = case_opinion.append(temp, ignore_index=True)

In [35]:
case_opinion['date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in case_opinion['date']]

In [36]:
case_opinion.head(2)

,date,case_number,description,language,country,venue,abstract,procedure_type,authority,ecli


In [37]:
tuples = to_tuples(case_opinion)

In [38]:
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `case_opinion`"
                "(date, case_number, description, language, country, venue, abstract, procedure_type, authority, ecli)"
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
        cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('case_opinion added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

case_opinion added


---
### Legal Intelligence Cases

In [39]:
legal_intelligence_case = pd.DataFrame()
legal_intelligence_case_columns = ['ecli',
                                   'name',
                                   'date',
                                   'abstract',
                                   'subject',
                                   'link',
                                   'DisplayTitle',
                                   'OriginalUrl',
                                   'Jurisidiction',
                                   'DocumentType',
                                   'CaseNumber',
                                   'PublicationNumber',
                                   'IssueNumber',
                                   'lodge_date',
                                   'DateAdded',
                                   'Sources',
                                   'UrlWithAutoLogOnToken',
                                   'court',
                                   'DisplaySubtitle']

In [40]:
temp = df_li_cases[['ecli',
                    'Title',
                    'date',
                    'abstract',
                    'LawArea',
                    'Url',
                    'DisplayTitle',
                    'OriginalUrl',
                    'Jurisdiction',
                    'DocumentType',
                    'case_number',
                    'PublicationNumber',
                    'IssueNumber',
                    'lodge_date',
                    'DateAdded',
                    'Sources',
                    'UrlWithAutoLogOnToken',
                    'authority',
                    'DisplaySubtitle']]
temp.columns = legal_intelligence_case_columns
legal_intelligence_case = legal_intelligence_case.append(temp, ignore_index=True)

In [41]:
legal_intelligence_case.head(2)

,ecli,name,date,abstract,subject,link,DisplayTitle,OriginalUrl,Jurisidiction,DocumentType,CaseNumber,PublicationNumber,IssueNumber,lodge_date,DateAdded,Sources,UrlWithAutoLogOnToken,court,DisplaySubtitle
0,ECLI:NL:RVS:2005:AS8403,"Gevaarlijke afvalstoffen, Enkel laten liggen, ...",19990101,"Gevaarlijke afvalstoffen, Enkel laten liggen, ...",['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/728...,"JAF 1999/15 - Met noot - Meijden van der, D. -...",NULL,Nederland,Rechtspraak,200404816/1,JAF 1999/15,JAF/1999/000001,20050302,20120822,JAF,http://www.legalintelligence.com/documents/728...,Raad van State,"Gevaarlijke afvalstoffen, Enkel laten liggen, ..."
1,ECLI:NL:RVS:2000:AL2818,Casus Burgemeester en wethouders van Wageninge...,20000101,Casus Burgemeester en wethouders van Wageninge...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/497...,Casus Burgemeester en wethouders van Wageninge...,https://www.navigator.nl/#/document/daee57d5fb...,Nederland,Rechtspraak,199902654/2 ECLI:NL:RVS:2000:AL2818,NULL,NULL,20000328,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/497...,Raad van State,"Nijhoff, H.P. - Milieurecht Totaal - Module-se..."


In [42]:
#legal_intelligence_case['name'] = [i[0:250] for i in legal_intelligence_case['name']]
#legal_intelligence_case['DisplayTitle'] = [i[0:250] for i in legal_intelligence_case['DisplayTitle']]
legal_intelligence_case['date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_cases['date']]
legal_intelligence_case['lodge_date'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_cases['lodge_date']]
legal_intelligence_case['DateAdded'] = [pd.to_datetime(i, errors='coerce') if i != 'NULL' else pd.to_datetime('1900-01-01 00:00:00') for i in df_li_cases['DateAdded']]

In [43]:
tuples = to_tuples(legal_intelligence_case)

In [44]:
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `legal_intelligence_case`"
                "(ecli, name, date, abstract, subject, link, DisplayTitle, OriginalUrl, Jurisdiction, DocumentType, CaseNumber, PublicationNumber, IssueNumber, lodge_date, DateAdded, Sources, UrlWithAutoLogOnToken, court, DisplaySubtitle)"
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
        cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('legal intelligence added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

legal intelligence added


---
### Subjects

In [45]:
subjects_as_list = [list(row.split("; ")) for row in df_case.subject]
unique_subjects = \
    set(list(
        pd.core.common\
            .flatten(subjects_as_list)))

In [46]:
subject = pd.DataFrame()
subject['name'] = list(sorted(unique_subjects))
subject.loc[:,'standard_name'] = 'NULL'

In [47]:
subject.head(2)

,name,standard_name
0,Aanbestedingsrecht,NULL
1,Ambtenarenrecht,NULL


In [48]:
#clean_table_sql('subject')

In [49]:
tuples = to_tuples(subject)

In [50]:
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `subject`"
                "(name, standard_name)"
                "VALUES (%s, %s)")
        #print(idx)
        cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('subject added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

subject added


---
### Case - Subject

In [51]:
df_subjects_case = df_case[['subject','case_id']]
df_subjects_case.loc[:,'subject'] = subjects_as_list
df_subjects_case = df_subjects_case.explode('subject')
len(df_subjects_case)

277668

In [52]:
parents_ids_subjects = get_parent_ids('subject', 'name', df_subjects_case, 'subject')
len(parents_ids_subjects)

277668

In [53]:
parents_ids_cases = get_parent_ids('case', 'ecli', df_subjects_case, 'case_id')
len(parents_ids_cases)

277668

In [54]:
case_subject = pd.DataFrame({'case_id':parents_ids_cases,
                             'subject_id':parents_ids_subjects})

In [55]:
#case_subject.sort_values(by='case_id').tail(50) #quality check
#[(type(i), type(j)) for i,j in zip(case_subject.subject_id,case_subject.case_id)]

In [56]:
case_subject.tail(3)

,case_id,subject_id
277665,199998,21
277666,199999,21
277667,200000,21


In [57]:
tuples = to_tuples(case_subject)
tuples = [(int(i), int(j)) for i,j in tuples]

In [58]:
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `case_subject`"
                "(case_id, subject_id)"
                "VALUES (%s, %s)")
        cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('case_subject added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

case_subject added


---
### Countries

In [59]:
df_country.head()

,country_id,eu,name,flag
0,AT,1,Austria,https://www.countryflags.io/at/flat/64.png
1,BE,1,Belgium,https://www.countryflags.io/be/flat/64.png
2,BG,1,Bulgaria,https://www.countryflags.io/bg/flat/64.png
3,CH,0,Switzerland,https://www.countryflags.io/ch/flat/64.png
4,CZ,1,Czechia,https://www.countryflags.io/cz/flat/64.png


In [60]:
df_country.loc[:,'language'] = 'NULL'
df_country.loc[:,'eea'] = 0

In [61]:
country = pd.DataFrame()
country_columns = ['id',
                   'name',
                   'language',
                   'flag',
                   'eu',
                   'eea']

In [62]:
temp = df_country[['country_id',
                   'name',
                   'language',
                   'flag',
                   'eu',
                   'eea']]
temp.columns = country_columns
country = country.append(temp, ignore_index=True)

In [63]:
country.head(3)

,id,name,language,flag,eu,eea
0,AT,Austria,NULL,https://www.countryflags.io/at/flat/64.png,1,0
1,BE,Belgium,NULL,https://www.countryflags.io/be/flat/64.png,1,0
2,BG,Bulgaria,NULL,https://www.countryflags.io/bg/flat/64.png,1,0


In [64]:
tuples = to_tuples(country)

In [65]:
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `country`"
                "(id, name, language, flag, eu, eea)"
                "VALUES (%s, %s, %s, %s, %s, %s)")
        cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('country added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

country added


---
### Case - Country

In [66]:
df_case.head(2)

,case_id,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,...,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text
0,ECLI:NL:CBB:2000:ZG2046,2000-01-11,95/0775/062/229,NULL,NL,NULL,NULL,NULL,NULL,NL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,ECLI:NL:CBB:2000:ZG1822,2000-02-15,98/280;20100,NULL,NL,NULL,NULL,NULL,NULL,NL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [67]:
#as_list = [list(row.split("; ")) for row in df_case.country] #in case there is more than one country in the row

In [68]:
df_country_case = df_case[['case_id','country']]
#df_country_case.loc[:,'country'] = as_list
df_country_case = df_country_case.explode('country')
len(df_country_case)

200000

In [69]:
parents_ids_countries = get_parent_ids('country', 'id', df_country_case, 'country')
len(parents_ids_countries)

200000

In [70]:
parents_ids_cases = get_parent_ids('case', 'ecli', df_country_case, 'case_id')
len(parents_ids_cases)

200000

In [71]:
case_country = pd.DataFrame({'case_id':parents_ids_cases,
                             'country_id':parents_ids_countries})

In [72]:
case_country.tail(3)

,case_id,country_id
199997,199998,NL
199998,199999,NL
199999,200000,NL


In [73]:
tuples = to_tuples(case_country)
#tuples = [(int(i), int(j)) for i,j in tuples]

In [74]:
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `case_country`"
                "(case_id, country_id)"
                "VALUES (%s, %s)")
        cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('case_subject added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

case_subject added


---
### Case law citation

In [75]:
df_case_citation.head(4)

,source_ecli,source_paragraph,target_ecli,target_paragraph


In [76]:
parent_ids = get_parent_ids('case', 'ecli', df_case_citation, 'source_ecli')
df_case_citation['case_id'] = parent_ids

In [77]:
case_citation = pd.DataFrame()
case_citation_columns = ['source_ecli',
                         'source_paragraph',
                         'target_ecli',
                         'target_paragraph',
                        'case_id']

In [78]:
temp = df_case_citation[['source_ecli',
                         'source_paragraph',
                         'target_ecli',
                         'target_paragraph',
                         'case_id']]
temp.columns = case_citation_columns
case_citation = case_citation.append(temp, ignore_index=True)

In [79]:
case_citation.head(3)

,source_ecli,source_paragraph,target_ecli,target_paragraph,case_id


In [80]:
tuples = to_tuples(case_citation)

In [81]:
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `case_citation`"
                "(source_ecli, source_paragraph, target_ecli, target_paragraph, case_id)"
                "VALUES (%s, %s, %s, %s, %s)")
        cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('case_citation added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

case_citation added


---
### Legislation citation

In [82]:
df_legislation_citation.head(4)

,source_ecli,source_paragraph,target_article,target_article_paragraph,target_article_webpage


In [83]:
parent_ids = get_parent_ids('case', 'ecli', df_legislation_citation, 'source_ecli')
df_legislation_citation['case_id'] = parent_ids

In [84]:
df_legislation_citation.loc[:,'target_name'] = 'NULL'
df_legislation_citation.loc[:,'target_sourcename'] = 'NULL'

ValueError: cannot set a frame with no defined index and a scalar

In [ ]:
legislation_citation = pd.DataFrame()
legislation_citation_columns = ['source_ecli',
                                'source_paragraph',
                                'target_id',
                                'target_paragraph',
                                'target_name',
                                'target_sourcename',
                                'target_link',
                                'case_id']

In [ ]:
temp = df_legislation_citation[['source_ecli',
                                'source_paragraph',
                                'target_article',
                                'target_article_paragraph',
                                'target_name',
                                'target_sourcename',
                                'target_article_webpage',
                                'case_id']]
temp.columns = legislation_citation_columns
legislation_citation = legislation_citation.append(temp, ignore_index=True)

In [ ]:
legislation_citation.head(3)

In [ ]:
tuples = to_tuples(legislation_citation)

In [ ]:
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `legislation_citation`"
                "(source_ecli, source_paragraph, target_id, target_paragraph, target_name, target_sourcename, target_link, case_id)"
                "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)")
        cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('legislation_citation added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

---
### Case related decision

In [ ]:
df_case_related = df_case[['case_id','related_cases']]\
    .rename(columns = {'case_id':'source_ecli', 
                       'related_cases': 'referencing_case_ecli'})

In [ ]:
parent_ids = get_parent_ids('case', 'ecli', df_case_related, 'source_ecli')
df_case_related['case_id'] = parent_ids

In [ ]:
df_case_related.loc[:,'referenced_case_ecli'] = 'NULL'

In [ ]:
df_case_related.head(2)

In [ ]:
case_related_decision = pd.DataFrame()
case_related_decision_columns = ['source_ecli',
                                 'referencing_case_ecli',
                                'referenced_case_ecli',
                                'case_id']

In [ ]:
temp = df_case_related[['source_ecli',
                                'referencing_case_ecli',
                                'referenced_case_ecli',
                                'case_id']]
temp.columns = case_related_decision_columns
case_related_decision = case_related_decision.append(temp, ignore_index=True)

In [ ]:
case_related_decision.head(3)

In [ ]:
tuples = to_tuples(case_related_decision)

In [ ]:
try:  
    cursor = conn.cursor()
    for idx, data in enumerate(tuples):
        query = ("INSERT INTO `case_related_decision`"
                "(source_ecli, referencing_case_ecli, referenced_case_ecli, case_id)"
                "VALUES (%s, %s, %s, %s)")
        cursor.execute(query, data)
    conn.commit()
    cursor.close()
    print('case_related_decision added')
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)